![](./resources/System_v1_training_header.png)

This notebook contains a demonstration on how to train custom crop type models based on your own reference data and how to apply the resulting model to generate a custom crop type map.

# Content

- [Before you start](#before-you-start)
- [1. Define region of interest](#1.-Define-a-region-of-interest)
- [2. Check available in-situ reference data](#2.-Check-available-in-situ-reference-data)
- [3. Prepare own reference data](#3.-Prepare-own-reference-data)
- [4. Train crop type classification model](#4.-Train-crop-type-classification-model)
- [5. Generate crop type map](#5.-Generate-crop-type-map)

# Before you start

In order to run this notebook, you need to create an account on:

- The Copernicus Data Space Ecosystem (CDSE)
--> by completing the form [HERE](https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=eRKGqDvoYI0)

- VITO's Terrascope platform
--> by completing the form [HERE](https://sso.terrascope.be/auth/realms/terrascope/login-actions/registration?client_id=drupal-terrascope&tab_id=irBzckp2aDo)

In [1]:
# First we import the necessary modules to run this notebook

import pandas as pd
import geopandas as gpd
import requests
from pathlib import Path
from shapely.geometry import shape, Polygon
import xarray as xr

import openeo
from openeo_gfmap import BoundingBoxExtent, TemporalContext
from openeo_gfmap.backend import Backend, BackendContext

import sys
sys.path.append('/home/jovyan/worldcereal-classification/src')

from worldcereal.utils.map import (get_ui_map, _latlon_to_utm)
from worldcereal.utils.refdata import _to_points
from worldcereal.utils.wrapper import run_inference

RDM_API = 'https://ewoc-rdm-api.iiasa.ac.at'

# 1. Define a region of interest

When running the code snippet below, an interactive map will be visualized.
Click the Rectangle button on the left hand side of the map to start drawing your region of interest.
When finished, execute the second cell to store the coordinates of your region of interest. 

In [2]:
m, dc = get_ui_map()
m

Map(center=[51.1872, 5.1154], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [3]:
# retrieve bounding box from drawn rectangle
obj = dc.last_draw
if obj.get('geometry') is not None:
    poly = Polygon(shape(obj.get('geometry')))
    bbox = poly.bounds
else:
    raise ValueError('Please first draw a rectangle '
                     'on the map before proceeding.')
print(f'Your area of interest: {bbox}')

Your area of interest: (4.182795, 51.275179, 4.217471, 51.294719)


# 2. Check available in situ reference data

xxx